In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from pathlib import Path
import pickle
from exported import mean_pooling

In [ ]:
upstream = ['embedding', 'load', 'validation']
product = None

In [ ]:
embeddings_reduced = pickle.loads(Path(upstream['embedding']['embeddings_reduced']).read_bytes())
rev_df = pickle.loads(Path(upstream['embedding']['rev_df']).read_bytes())
FEATURE_COLUMNS = pickle.loads(Path(upstream['load']['FEATURE_COLUMNS']).read_bytes())
MISSING_COLUMNS = pickle.loads(Path(upstream['validation']['MISSING_COLUMNS']).read_bytes())

## Testing

In [9]:
# Predict rating
X = np.hstack(
    [rev_df[FEATURE_COLUMNS + MISSING_COLUMNS].to_numpy(), embeddings_reduced]
)
y = rev_df['rating']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# XGBoost
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_model.fit(X_train, y_train)

# Evaluate
xgb_model.score(X_test, y_test)

0.9722471833229065

In [ ]:
Path(product['xgb_model']).parent.mkdir(exist_ok=True, parents=True)
Path(product['xgb_model']).write_bytes(pickle.dumps(xgb_model))